# Test Notebook 

In [1]:
# Importing Relevant Libraries
import os, os.path
import importlib
import itertools
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.stats import qmc
import pyDOE2 as pyd
import seaborn as sns
from matplotlib.pyplot import plot, savefig
import time
import datetime 
from datetime import date
from pyomo.opt import SolverStatus, TerminationCondition
import matplotlib.ticker as mtick
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
import graphviz 
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.patches as mpatches
import matplotlib.lines as mlines
import logging
from pyomo.environ import *
import yaml
from pathlib import Path

# Source: https://danshiebler.com/2016-09-14-parallel-progress-bar/
from tqdm import tqdm
import concurrent.futures
from concurrent.futures import ProcessPoolExecutor, as_completed
import sys # Python "sys" documentation: https://docs.python.org/3/library/sys.html

# To reset the default parameters of matplotlib:
import matplotlib as mpl

## Creating Future Samples

In [108]:
import creating_futures_sample
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [132]:
with open('config.yaml', 'r') as f:
    config = yaml.safe_load(f)
config

{'input_file_path': 'PortfolioFiles/Portfolio_Files_Latest/All data.xlsx',
 'n_sample': 10,
 'sampling_method': 'size',
 'future_zero': True,
 'output_string': 'checking',
 'output_path': 'PortfolioFiles/sample_of_futures/',
 'output_type': 'csv',
 'sampling_size': {'size': {'Energy': 4,
   'Land': 3,
   'ER': 10,
   'Ymax': 10,
   'GeoSt': 5},
  'not_sampled': {'OPEX': '-', 'SCref': '-', 'Yref': '-', 'Xmax': '-'},
  'exceptions': {'CDRRequired': 100000000000,
   'EnergyL': 5,
   'LandL': 100000000}},
 'sampling_step': {'step': {'Energy': 0.5,
   'Land': 0.01,
   'ER': 0.01,
   'Ymax': 500000000,
   'GeoSt': 4000000000,
   'CDRRequired': 100000000000,
   'EnergyL': 5,
   'LandL': 100000000},
  'not_sampled': {'OPEX': '-', 'SCref': '-', 'Yref': '-', 'Xmax': '-'}}}

In [ ]:
input_file_path = Path(config['input_file_path'])

n_sample = config['n_sample']
dict_constant = {} # Missing to configure
sampling_method = (config['sampling_method'], config[f"sampling_{config['sampling_method']}"])
future_zero = config['future_zero']
output_file = f"{config['output_path']}{config['output_string']}_{'OA'}_{date.today().strftime('%d.%m.%Y')}.{config['output_type']}"
output_type = config['output_type']

attempt = creating_futures_sample.sampling_futures(input_file_path, n_sample, dict_constant, sampling_method, future_zero, output_file, output_type)
df, units, sampled_parameters = attempt.sample()
# If FutureWarning error: ignore, as might be a bug: https://github.com/pandas-dev/pandas/issues/55025

df

c:\Users\rodq\Documents\Python Scripts\git_projects\CDR_Portfolios\creating_futures_sample.py:167: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '6.467' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_LHS.loc[futureID,parameter] = round(self.df.loc[parameter,"minimum"] + (df_LHS.loc[futureID,parameter]-1)*self.df_Sampling_step.loc[parameter,"Sampling_step"], 3)
c:\Users\rodq\Documents\Python Scripts\git_projects\CDR_Portfolios\creating_futures_sample.py:167: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '6.367' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_LHS.loc[futureID,parameter] = round(self.df.loc[parameter,"minimum"] + (df_LHS.loc[futureID,parameter]-1)*self.df_Sampling_step.loc[parameter,"Sampling_step"], 3)
c:\Users\rodq\Docume

,Energy_DACCS,Energy_DOCCS,Energy_EW,Energy_OA,EnergyL,Land_A/R,Land_BECCS,LandL,ER_A/R,ER_BC,...,Yref_OA,Yref_SCS,Xmax_A/R,Xmax_BC,Xmax_BECCS,Xmax_DACCS,Xmax_DOCCS,Xmax_EW,Xmax_OA,Xmax_SCS
future_id,,,,,,,,,,,,,,,,,,,,,
0,4.700,2.500,0.5,0.700,2030.0,0.07,0.040,2.000000e+08,-0.230,-0.050,...,1.0,1.0,80000000.0,500000000.0,500000000.0,500000000.0,500000000.0,1.000000e+09,1.000000e+09,1.000000e+09
1,6.467,6.367,12.5,6.800,2040.0,0.09,0.170,2.000000e+08,-0.036,0.000,...,1.0,1.0,80000000.0,500000000.0,500000000.0,500000000.0,500000000.0,1.000000e+09,1.000000e+09,1.000000e+09
2,6.467,2.500,8.5,2.733,2035.0,0.07,0.105,3.000000e+08,-0.063,0.050,...,1.0,1.0,80000000.0,500000000.0,500000000.0,500000000.0,500000000.0,1.000000e+09,1.000000e+09,1.000000e+09
3,8.233,6.367,8.5,2.733,2040.0,0.07,0.170,9.000000e+08,-0.091,-0.017,...,1.0,1.0,80000000.0,500000000.0,500000000.0,500000000.0,500000000.0,1.000000e+09,1.000000e+09,1.000000e+09
4,10.000,8.300,12.5,0.700,2050.0,0.09,0.040,7.000000e+08,-0.147,-0.033,...,1.0,1.0,80000000.0,500000000.0,500000000.0,500000000.0,500000000.0,1.000000e+09,1.000000e+09,1.000000e+09
5,8.233,2.500,4.5,6.800,2050.0,0.07,0.105,4.000000e+08,-0.008,0.033,...,1.0,1.0,80000000.0,500000000.0,500000000.0,500000000.0,500000000.0,1.000000e+09,1.000000e+09,1.000000e+09
6,10.000,4.433,0.5,4.767,2035.0,0.07,0.170,8.000000e+08,-0.174,0.017,...,1.0,1.0,80000000.0,500000000.0,500000000.0,500000000.0,500000000.0,1.000000e+09,1.000000e+09,1.000000e+09
7,4.700,4.433,0.5,6.800,2030.0,0.08,0.040,3.000000e+08,-0.202,0.083,...,1.0,1.0,80000000.0,500000000.0,500000000.0,500000000.0,500000000.0,1.000000e+09,1.000000e+09,1.000000e+09
8,8.233,2.500,0.5,2.733,2030.0,0.09,0.105,1.000000e+09,-0.230,0.067,...,1.0,1.0,80000000.0,500000000.0,500000000.0,500000000.0,500000000.0,1.000000e+09,1.000000e+09,1.000000e+09


## Solving Futures

In [1]:
import Model_Solving_Futures_Class
%reload_ext autoreload
%autoreload 2

In [2]:
import yaml
with open('key_parameters.yaml', 'r') as f:
    key_parameters = yaml.safe_load(f)
key_parameters

{'i': ['BECCS', 'A/R', 'SCS', 'BC', 'DACCS', 'EW', 'OA', 'DOCCS'],
 'k1': 0,
 'delK': 5,
 'N': 4,
 'first_year': 2020,
 'last_year': 2100,
 'T': 'int(1 + (last_year-first_year)/delK)',
 'k': 'list(range(k1,T))',
 's': {'BECCS': 1,
  'A/R': 0,
  'SCS': 0,
  'BC': 0,
  'DACCS': 1,
  'EW': 0,
  'OA': 0,
  'DOCCS': 1},
 'rE': 0.03,
 'LTIni': {'BECCS': 10,
  'A/R': 25,
  'SCS': 5,
  'BC': 10,
  'DACCS': 10,
  'EW': 5,
  'OA': 5,
  'DOCCS': 5},
 'LT': {'BECCS': 20,
  'A/R': 55,
  'SCS': 20,
  'BC': 25,
  'DACCS': 25,
  'EW': 10,
  'OA': 20,
  'DOCCS': 25},
 'k_logi': 0.5,
 'L': 600000000000,
 't0': 10,
 'DACCS_available': 2030,
 'DOCCS_available': 2040,
 'OA_available': 2050,
 'color_map': 'Set2',
 'color_per_CDR': {'BECCS': 0.8,
  'A/R': 0.1,
  'SCS': 0.9,
  'BC': 0.2,
  'DACCS': 0.7,
  'EW': 0.3,
  'OA': 0.4,
  'DOCCS': 0.5}}

In [ ]:
from pathlib import Path
from datetime import date

n_evaluate = 'all'

futures_file = Path('PortfolioFiles/sample_of_futures/manually_selecting_10.12.2024.csv')
#futures_file = Path('PortfolioFiles\sample_of_futures\File Name_15.08.2024.xlsx')
extra_resources = Path('PortfolioFiles/Portfolio_Files_Latest/extra_resources.xlsx')

config = {'output_string': "aggregated_results", 
          'output_path': 'PortfolioFiles/results_from_modelling/',
          'output_type': "csv"} # xlsx or csv

output_file = f"{config['output_path']}{config['output_string']}_{date.today().strftime('%d.%m.%Y')}.{config['output_type']}"

attempt = Model_Solving_Futures_Class.model_solving_futures(key_parameters, futures_file, n_evaluate, extra_resources, output_file, num_jobs=10, front_solve=3)
attempt.solving_with_parallel()

Reading main :)
Simulation started at 14:08:06. Generating data input.
Done with preparing input data - let's start solving for futures at: 14:09:27
ERROR: Unable to clone Pyomo component attribute. Component 'Cont_x_index'
contains an uncopyable field '_init_dimen' (<class
'pyomo.core.base.initializer.ConstantInitializer'>).  Setting field to `None`
on new object
Optimal solution for future ID 0 found after 2.8 seconds.
model.name="unknown";
    - termination condition: infeasible
    - message from solver: Model was proven to be infeasible.
Solution for future ID 1 NOT found. Solver status is warning. Termination condition is infeasible (1.5 seconds elapsed).
ERROR: Unable to clone Pyomo component attribute. Component
'Upper_bound_lamb_index' contains an uncopyable field '_init_dimen' (<class
'pyomo.core.base.initializer.ConstantInitializer'>).  Setting field to `None`
on new object
Optimal solution for future ID 2 found after 1.9 seconds.


100%|██████████| 2998/2998 [00:00<00:00, 107425.90it/s]
2998it [00:00, 179188.08it/s]


Optimisation finished at 15:00:09. Elapsed time: 3042 seconds (51 minutes). Out of 3001 futures evaluated, 2687 have been solved. Starting data saving.


c:\Users\rodq\Documents\Python Scripts\git_projects\CDR_Portfolios\Model_Solving_Futures_Class.py:727: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aggregated_results['Solved'] = pd.Series(solved_binary)


Data has been saved at: 15:11:34


,Energy_DACCS,Energy_DOCCS,Energy_EW,Energy_OA,EnergyL,Land_A/R,Land_BECCS,LandL,ER_A/R,ER_BC,...,OA_2100,DOCCS_2100,BECCS_removed,A/R_removed,SCS_removed,BC_removed,DACCS_removed,EW_removed,OA_removed,DOCCS_removed
future_id,,,,,,,,,,,,,,,,,,,,,
0,4.700,2.500,0.5,0.700,2030,0.07,0.040,200000000,-0.230,-0.050,...,6.982525e+08,0.0,27300000.0,150000000000.0,33000000010.0,10507500000.0,172500149995.0,20000000010.0,13965049975.000002,10.0
1,4.700,8.300,12.5,4.767,2045,0.08,0.170,1000000000,-0.230,-0.033,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10.000,8.300,0.5,2.733,2045,0.08,0.170,600000000,-0.230,0.083,...,0.000000e+00,0.0,27300000.0,193679167114.034058,116600000010.0,90907499990.0,98786032855.966003,10.0,10.0,10.0
3,6.467,2.500,4.5,6.800,2030,0.07,0.170,400000000,-0.202,0.100,...,1.000000e+09,0.0,66363880893.392693,182199999995.0,87750000010.0,225507499990.0,370804785538.509277,49521632721.333359,17852200841.764668,10.0
4,10.000,6.367,12.5,2.733,2030,0.07,0.040,600000000,-0.063,0.017,...,1.080000e+10,0.0,114195515775.0,279399999945.0,62000000010.0,23257500000.0,153963848251.971924,10.0,267183135998.028137,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2996,8.233,2.500,8.5,2.733,2030,0.09,0.105,700000000,-0.202,0.017,...,5.200000e+09,0.0,96005677594.192337,279399999945.0,159000000010.0,153807499995.0,297048027570.984253,648498313.607637,114090296561.215927,10.0
2997,6.467,2.500,12.5,0.700,2050,0.08,0.040,300000000,-0.174,-0.050,...,3.800000e+09,0.0,89705912812.308456,221326911675.689484,159000000010.0,41857500000.0,119777588750.898193,7040104549.675318,161291982191.428589,10.0
2998,10.000,6.367,0.5,4.767,2050,0.09,0.105,200000000,0.020,-0.017,...,1.220000e+10,0.0,27300000.0,162499839534.237,99999999990.000015,39007500000.0,14879223348.756834,10811545575.806545,172774591541.199585,10.0
